In [ ]:
def bsm_call_value (S, X, t, r, sigma):
  '''
  S : float - initial stock/index level
  X : float - strike price
  t : float - maturity date
  r : float - constant risk-free short rate
  sigma : float - volatility
  '''

  from math import log, sqrt, exp
  from scipy import stats
  
  S = float(S)
  d1 = (log(S/X) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
  d2 = (log(S/X) + (r - 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
  value = (S * stats.norm.cdf(d1, 0.0, 1.0) - X * exp (-r * t) * stats.norm.cdf(d2, 0.0, 1.0))
  return value

In [ ]:
def bsm_vega(S, X, t, r, sigma): 
  '''
  S : float - initial stock/index level 
  X : float - strike price 
  t : float - maturity date
  r : float - constant risk-free short rate 
  sigma : float - volatility factor
  ''' 

  from math import log, sqrt 
  from scipy import stats 
  S = float(S) 
  d1 = (log(S/X) + (r + 0.5 * sigma ** 2) * t) / (sigma * sqrt(t))
  vega = S * stats.norm.cdf(d1, 0.0, 1.0) * sqrt(t)
  return vega

In [ ]:
def bsm_call_imp_vol(S, X, t, r, C, sigma_est, it = 100): 
  ''' 
  S : float - initial stock/index level 
  X : float - strike price 
  t : float - maturity date
  r : float - constant risk-free short rate 
  C : float - option level
  sigma_est : float estimate of impl. volatility 
  it : integer number of iterations 
  '''

  for i in range (it): 
    sigma_est -= ((bsm_call_value(S, X, t, r, sigma_est) - C) / bsm_vega(S, X, t, r, sigma_est))
  return sigma_est

In [ ]:
from dao import InvestDao
dao = InvestDao()

In [ ]:
r = dao.sql("select rate from rates where dtyymmdd='2017-06-02' and symbol='WIBOR6M' order by dtyymmdd")[0]
rate = float(r[0])/100.0

In [ ]:
(ttm, strike, close) = dao.sql("select ttm, strike, close from v_option_quotes where ticker='OW20C181800' and dtyymmdd='2017-06-02' and type='C' order by dtyymmdd")[0]

In [ ]:
(initial, avg_close, std_close) = dao.sql("select close, avg_close, std_close from v_stock_stats where dtyymmdd = '2017-06-02' order by dtyymmdd")[0]
volatility = float(std_close)/float(avg_close)

In [ ]:
implied_volatility = bsm_call_imp_vol(float(initial), float(strike), float(ttm), float(rate), float(close), volatility)

In [ ]:
print("implied volatility = %s" % implied_volatility)